### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data = "Resources/schools_complete.csv"
student_data = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_df = pd.read_csv(school_data)
student_df = pd.read_csv(student_data)

# Combine the data into a single dataset.  
complete_data = pd.merge(student_df, school_df, how="left", on=["school_name", "school_name"])

In [2]:
complete_data.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [3]:
#total number of schools: 15
unique_school_name = school_df['school_name'].unique()
total_school = len(unique_school_name)
# count_student 'size' :39170   # count_student = school_df['size'].sum()
total_student = student_df['student_name'].count()

# total budget
total_budget = school_df['budget'].sum()
# average math score 78.98
avg_math_score = complete_data['math_score'].mean()
# average reading score 81.87
avg_read_score = complete_data['reading_score'].mean()

# Calculate the percentage of students with a passing math score (70 or greater) 29370 : 74.98%
student_pass_math = complete_data[(complete_data['math_score'] >=70)].count()["student_name"]
percent_pass_math = student_pass_math/float(total_student) *100
# Calculate the percentage of students with a passing reading score (70 or greater): 85.5% : 33610
student_pass_read = complete_data[(complete_data['reading_score'] >=70)].count()["student_name"]
percent_pass_read = student_pass_read/float(total_student) *100

# Overall passsing and percent: 65.17%
overall_pass = complete_data[(complete_data['math_score'] >=70) 
                             & (complete_data['reading_score'] >=70)]['student_name'].count()/total_student
percent_overall = overall_pass *100

district_summary = pd.DataFrame({"Total Schools": [total_school],
                                 "Total Students": [total_student],
                                 "Total Budget": [total_budget],
                                 "Average Math Score": [avg_math_score],
                                 "Average Reading Score": [avg_read_score],
                                 "% Passing Math": [percent_pass_math],
                                 "% Passing Reading": [percent_pass_read],
                                 "% Overall Passing":[percent_overall]})

district_summary = district_summary[["Total Schools", "Total Students", "Total Budget", "Average Math Score",
                   "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]]

format_summary = district_summary.style.format({"Total Students": "{:,}",
                                        "Total Budget": "${:,.2f}"}) 
                       
format_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.877840,74.980853,85.805463,65.172326


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [4]:
school_type = school_df.set_index(["school_name"])["type"]
##
student_per_school = complete_data["school_name"].value_counts()
budget_per_school = complete_data.groupby(["school_name"]).mean()["budget"]
# budget per student
per_student_budget = budget_per_school/ student_per_school
# avg
avg_math_score = complete_data.groupby(["school_name"]).mean()['math_score']
avg_read_score = complete_data.groupby(["school_name"]).mean()['reading_score']
## percent_pass_math 
student_pass_math = complete_data[(complete_data['math_score'] >=70)]
student_pass_math_school = student_pass_math.groupby(["school_name"]).count()["student_name"]
percent_math = (student_pass_math_school/student_per_school)*100
# percent_math
## percent_pass_read 
student_pass_read = complete_data[(complete_data['reading_score'] >=70)]
student_pass_read_school = student_pass_read.groupby(["school_name"]).count()["student_name"]
percent_read = (student_pass_read_school/student_per_school)*100
# percent_read

#overall_passing
overall_passing = (complete_data[(complete_data['reading_score'] >=70) 
                    & (complete_data['math_score'] >=70)].groupby("school_name")['student_name'].count()/student_per_school)*100

school_summary = pd.DataFrame({"School Type":school_type, "Total Students":student_per_school,
                "Total School Budget":budget_per_school, "Per Student Budget":per_student_budget,
                "Average Math Score":avg_math_score, "Average Reading Score":avg_read_score,
                "% Passing Math":percent_math, "% Passing Reading":percent_read, 
                "% Overall Passing":overall_passing})
school_summary = school_summary[["School Type", "Total Students", "Total School Budget", "Per Student Budget",
                "Average Math Score", "Average Reading Score", "% Passing Math",
                "% Passing Reading", "% Overall Passing"]]

format_summary2 = school_summary.style.format({"Total School Budget": "${:,.2f}",
                                        "Per Student Budget": "${:,.2f}"}) 
format_summary2

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [23]:
# top_perform_schools
top_school = school_summary.sort_values("% Overall Passing", ascending = False)
top_school.head().style.format({'Total Students': '{:,}',
                           "Total School Budget": "${:,}", 
                           "Per Student Budget": "${:.0f}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,"1,858","$1,081,356.0",$582,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,"1,635","$1,043,130.0",$638,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,"1,468","$917,500.0",$625,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,"2,283","$1,319,574.0",$578,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.0",$609,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [24]:
bottom_school = top_school.tail()
bottom_school = bottom_school.sort_values('% Overall Passing')
bottom_school.style.format({'Total Students': '{:,}', 
                       "Total School Budget": "${:,}", 
                       "Per Student Budget": "${:.0f}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,"3,999","$2,547,363.0",$637,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,"2,949","$1,884,411.0",$639,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,"2,917","$1,910,635.0",$655,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,"4,635","$3,022,020.0",$652,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,"4,761","$3,094,650.0",$650,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [40]:
ninth_math = student_df.loc[student_df['grade'] == '9th'].groupby('school_name')['math_score'].mean()
tenth_math = student_df.loc[student_df['grade'] == '10th'].groupby('school_name')['math_score'].mean()
eleventh_math = student_df.loc[student_df['grade'] == '11th'].groupby('school_name')['math_score'].mean()
twelfth_math = student_df.loc[student_df['grade'] == '12th'].groupby('school_name')['math_score'].mean()

math_scores = pd.DataFrame({'9th':ninth_math, '10th':tenth_math, 
                            '11th':eleventh_math, '12th':twelfth_math})
math_scores = math_scores[['9th', '10th', '11th', '12th']]

math_scores

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [39]:
ninth_grade = student_df.loc[student_df['grade']=='9th'].groupby('school_name')['reading_score'].mean()
tenth_grade = student_df.loc[student_df['grade']=='10th'].groupby('school_name')['reading_score'].mean()
eleventh_grade = student_df.loc[student_df['grade']=='11th'].groupby('school_name')['reading_score'].mean()
twelth_grade = student_df.loc[student_df['grade']=='12th'].groupby('school_name')['reading_score'].mean()

reading_score = pd.DataFrame({'9th':ninth_grade, '10th':tenth_grade,
                   '11th':eleventh_grade ,'12th':twelth_grade})
reading_score = reading_score[['9th','10th','11th','12th']]

reading_score

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

#### *Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [80]:
# create size_bins # bins = [0, 584.99, 514.99, 644.99, 680]
bins = [0, 584.999, 614.999, 644.999, 999999]
group_name = ["< $585", "$585-630", "$630-645", "$645-680"]
complete_data['spending_bin'] = pd.cut(complete_data['budget']/complete_data['size'], bins, labels = group_name)

by_spending = complete_data.groupby('spending_bin')

avg_math = by_spending['math_score'].mean()
avg_read = by_spending['reading_score'].mean()
pass_math = (complete_data[complete_data['math_score']>=70].groupby('spending_bin')['student_name'].count()/by_spending['student_name'].count())*100
pass_read = (complete_data[complete_data['reading_score']>=70].groupby('spending_bin')['student_name'].count()/by_spending['student_name'].count())*100
overall = (complete_data[(complete_data['reading_score']>=70) & (complete_data['math_score']>=70)].groupby('spending_bin')['student_name'].count()/by_spending['student_name'].count())*100

score_by_spend = pd.DataFrame({"Average Math Score": avg_math, "Average Reading Score": avg_math,
                               "% Passing Math": pass_math, "% Passing Reading": pass_read,
                                "%Overall Passing": overall})

#formating
score_by_spend.style.format({'Average Math Score': '{:.1f}', 
                              'Average Reading Score': '{:.1f}', 
                              '% Passing Math': '{:.1%}', 
                              '% Passing Reading':'{:,2f}', 
                              '%Overall Passing': '{:,2f}'})
score_by_spend

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,%Overall Passing
spending_bin,,,,,
< $585,83.363065,83.363065,93.702889,96.686558,90.640704
$585-630,83.529196,83.529196,94.124128,95.886889,90.121190
$630-645,78.061635,78.061635,71.400428,83.614770,60.289317
$645-680,77.049297,77.049297,66.230813,81.109397,53.528791


## Scores by School Size

* Perform the same operations as above, based on school size.

In [87]:
# create size bins
bins = [0, 999, 1999, 99999999999]
group_name = ["Small (<1000)", "Medium (1000-2000)" , "Large (>2000)"]
complete_data['size_bins'] = pd.cut(complete_data['size'], bins, labels = group_name)

#group by spending
by_size = complete_data.groupby('size_bins')

#calculations 
avg_math = by_size['math_score'].mean()
avg_read = by_size['math_score'].mean()
pass_math = complete_data[complete_data['math_score'] >= 70].groupby('size_bins')['Student ID'].count()/by_size['Student ID'].count()
pass_read = complete_data[complete_data['reading_score'] >= 70].groupby('size_bins')['Student ID'].count()/by_size['Student ID'].count()
overall = complete_data[(complete_data['reading_score'] >= 70) & (complete_data['math_score'] >= 70)].groupby('size_bins')['Student ID'].count()/by_size['Student ID'].count()

            
           
scores_by_size = pd.DataFrame({
    "Average Math Score": avg_math, "Average Reading Score": avg_read,
    '% Passing Math': pass_math, '% Passing Reading': pass_read,
    "%Overall Passing": overall})
            
#list columns
score_by_size = scores_by_size[["Average Math Score", "Average Reading Score",
        '% Passing Math', '% Passing Reading', "%Overall Passing"]]

#formating
score_by_size.style.format({'Average Math Score': '{:.1f}', 
                              'Average Reading Score': '{:.1f}', 
                              '% Passing Math': '{:.1%}', 
                              '% Passing Reading':'{:.1%}', 
                              '%Overall Passing': '{:.1%}'})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,%Overall Passing
size_bins,,,,,
Small (<1000),83.8,83.8,94.0%,96.0%,90.1%
Medium (1000-2000),83.4,83.4,93.6%,96.8%,90.6%
Large (>2000),77.5,77.5,68.7%,82.1%,56.6%


## Scores by School Type

* Perform the same operations as above, based on school type

In [92]:
# group by type of school
by_type = complete_data.groupby("type")

#calculations 
avg_math = by_type['math_score'].mean()
avg_read = by_type['math_score'].mean()
pass_math = (complete_data[complete_data['math_score'] >= 70].groupby('type')['Student ID'].count()/by_type['Student ID'].count())*100
pass_read = (complete_data[complete_data['reading_score'] >= 70].groupby('type')['Student ID'].count()/by_type['Student ID'].count())*100
overall = (complete_data[(complete_data['reading_score'] >= 70) & (complete_data['math_score'] >= 70)].groupby('type')['Student ID'].count()/by_type['Student ID'].count())*100

           
score_by_type = pd.DataFrame({"Average Math Score": avg_math,"Average Reading Score": avg_read,
            '% Passing Math': pass_math,'% Passing Reading': pass_read,
            "%Overall Passing": overall})
    
#list columns
score_by_type = score_by_type[["Average Math Score","Average Reading Score",
        '% Passing Math','% Passing Reading',"%Overall Passing"]]

score_by_type.index.name = "Type of School"


#formating
score_by_type.style.format({'Average Math Score': '{:.1f}', 
                              'Average Reading Score': '{:.1f}'})


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,%Overall Passing
Type of School,,,,,
Charter,83.4,83.4,93.701821,96.645891,90.560932
District,77.0,77.0,66.518387,80.905249,53.695878
